# Internal consistency

### Imports

In [ ]:
import os
import re
import glob
import os
import ast
import os.path as op
from collections import defaultdict
from copy import deepcopy
import copy

import pickle
from time import time
import pywt
import mne
import scipy
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import cesium.featurize
from plotly.subplots import make_subplots
from ipywidgets import Dropdown, FloatRangeSlider, IntSlider, FloatSlider, interact
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.base import TransformerMixin, BaseEstimator

import sys

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.svm import SVR
from sklearn.decomposition import FastICA
from sklearn.decomposition import PCA

from rumination_experiment_transformers_averaged_CDS import *

import warnings

warnings.filterwarnings("ignore")


---
## Loading data

Loading EEG data and data from rumination questionnaire. By default create_df_data loads all info from given file but one can specify it by passing a list of desired labels from csv file.

In [ ]:
# paths TODO
dir_path = os.path.dirname(os.path.abspath(""))

In [ ]:
tmin, tmax = -0.1, 0.6  # Start and end of the segments
signal_frequency = 256
ERROR = 0
CORRECT = 1
ALL = 2
random_state = 0

In [ ]:
channels_order_list = [
    "Fp1",
    "AF7",
    "AF3",
    "F1",
    "F3",
    "F5",
    "F7",
    "FT7",
    "FC5",
    "FC3",
    "FC1",
    "C1",
    "C3",
    "C5",
    "T7",
    "TP7",
    "CP5",
    "CP3",
    "CP1",
    "P1",
    "P3",
    "P5",
    "P7",
    "P9",
    "PO7",
    "PO3",
    "O1",
    "Iz",
    "Oz",
    "POz",
    "Pz",
    "CPz",
    "Fpz",
    "Fp2",
    "AF8",
    "AF4",
    "AFz",
    "Fz",
    "F2",
    "F4",
    "F6",
    "F8",
    "FT8",
    "FC6",
    "FC4",
    "FC2",
    "FCz",
    "Cz",
    "C2",
    "C4",
    "C6",
    "T8",
    "TP8",
    "CP6",
    "CP4",
    "CP2",
    "P2",
    "P4",
    "P6",
    "P8",
    "P10",
    "PO8",
    "PO4",
    "O2",
]

channels_dict = dict(zip(channels_order_list, np.arange(1, 64, 1)))

In [ ]:
def create_df_data(
    test_participants=False,
    test_epochs=False,
    info_filename=None,
    info="all",
    personal=True,
):
    """Loads data for all participants and create DataFrame with optional additional info from given .csv file.

    On default, loads a train set: chooses only 80% of participants
    and for each of them chooses 80% of epochs.
    It will choose them deterministically.

    Participants with less than 10 epochs per condition are rejected.

    If test_participants is set to True, it will load remaining 20% of participants.
    If test_epochs is set to True, it will load remaining 20% of epochs.
    Test epochs are chronologically after train epochs,
    because it reflects real usage (first callibration and then classification).

    Parameters
    ----------
    test_participants: bool
        whether load data for training or final testing.
        If true load participants data for testing.
    test_epochs: bool
        whether load data for training or final testing.
        If true load epochs of each participants data for testing.
    info_filename: String | None
        path to .csv file with additional data.
    info: array
        listed parameters from the info file to be loaded.
        if 'all', load all parameters
    personal: bool
        whether a model will be both trained and tested on epochs from one person
        if false, person's epochs aren't split into test and train


    Returns
    -------
    go_nogo_data_df : pandas.DataFrame

    """
    print(os.path.abspath(""))
    dir_path = os.path.dirname(os.path.abspath(""))
    print(dir_path)
    header_files_glob = os.path.join(dir_path, "data/responses_100_600/*.vhdr")
    header_files = glob.glob(header_files_glob)

    header_files = sorted(header_files)
    go_nogo_data_df = pd.DataFrame()

    # cut 20% of data for testing
    h_train, h_test = train_test_split(header_files, test_size=0.3, random_state=0)

    if test_participants:
        header_files = h_test
    else:
        header_files = h_train

    for file in header_files:
        #  load eeg data for given participant
        participant_epochs = load_epochs_from_file(file)

        # and compute participant's id from file_name
        participant_id = re.match(r".*_(\w+).*", file).group(1)

        error = participant_epochs["error_response"]._data
        correct = participant_epochs["correct_response"]._data

        # exclude those participants who have too few samples
        if len(error) < 3 or len(correct) < 3:
            # not enough data for this participant
            continue

        # construct dataframe for participant with: id|epoch_data|response_type|additional info...
        participant_df = create_df_from_epochs(
            participant_id, participant_epochs, info_filename, info
        )
        print(participant_id)
        go_nogo_data_df = go_nogo_data_df.append(participant_df, ignore_index=True)

    return go_nogo_data_df

In [ ]:
def create_df_from_epochs(id, participant_epochs, info_filename, info):
    """Create df for each participant. DF structure is like: {id: String ; epoch: epoch_data ; marker: 1.0|0.0}
    1.0 means correct and 0.0 means error response.
    Default info extracted form .csv file is 'Rumination Full Scale' and participants' ids.
    With this info df structure is like:
    {id: String ; epoch: epoch_data ; marker: 1.0|0.0 ; File: id ; 'Rumination Full Scale': int}

    Parameters
    ----------
    id: String
        participant's id extracted from filename
    correct: array
        correct responses' data
    error: array
        error responses' data
    info_filename: String
        path to .csv file with additional data.
    info: array
        listed parameters from the info file to be loaded.
        if 'all', load all parameters

    Returns
    -------
    participant_df : pandas.DataFrame

    """
    participant_df = pd.DataFrame()
    info_df = pd.DataFrame()

    # get additional info from file
    if info_filename is not None:
        if info == "all":
            rumination_df = pd.read_csv(info_filename)
        else:
            rumination_df = pd.read_csv(info_filename, usecols=["File"] + info)
        info_df = (
            rumination_df.loc[rumination_df["File"] == id]
            .reset_index()
            .drop("index", axis=1)
        )

#     for epoch in correct:
#         epoch_df = pd.DataFrame(
#             {"id": [id], "epoch": [epoch], "marker": [CORRECT]}
#         ).join(info_df)
#         participant_df = participant_df.append(epoch_df, ignore_index=True)

#     for epoch in error:
#         epoch_df = pd.DataFrame({"id": [id], "epoch": [epoch], "marker": [ERROR]}).join(
#             info_df
#         )
#         participant_df = participant_df.append(epoch_df, ignore_index=True)
        
#     print(participant_epochs)
        
    epoch_df = pd.DataFrame({"id": [id], "epoch": [participant_epochs], "marker": [ALL]}).join(
            info_df
        )
    participant_df = participant_df.append(epoch_df, ignore_index=True)

    return participant_df

In [ ]:
def load_epochs_from_file(file, reject_bad_segments="auto", mask=None):
    """Load epochs from a header file.

    Args:
        file: path to a header file (.vhdr)
        reject_bad_segments: 'auto' means that bad segments are rejected automatically.

    Returns:
        mne Epochs

    """
    # Import the BrainVision data into an MNE Raw object
    raw = mne.io.read_raw_brainvision(file)

    # Construct annotation filename
    annot_file = file[:-4] + "vmrk"

    # Read in the event information as MNE annotations
    annotations = mne.read_annotations(annot_file)

    # Add the annotations to our raw object so we can use them with the data
    raw.set_annotations(annotations)

    # Map with response markers only
    event_dict = {
        "Stimulus/RE*ex*1_n*1_c_1*R*FB": 10004,
        "Stimulus/RE*ex*1_n*1_c_1*R*FG": 10005,
        "Stimulus/RE*ex*1_n*1_c_2*R": 10006,
        "Stimulus/RE*ex*1_n*2_c_1*R": 10007,
        "Stimulus/RE*ex*2_n*1_c_1*R": 10008,
        "Stimulus/RE*ex*2_n*2_c_1*R*FB": 10009,
        "Stimulus/RE*ex*2_n*2_c_1*R*FG": 10010,
        "Stimulus/RE*ex*2_n*2_c_2*R": 10011,
    }

    # Map for merged correct/error response markers
    merged_event_dict = {"correct_response": 0, "error_response": 1}

    # Reconstruct the original events from Raw object
    events, event_ids = mne.events_from_annotations(raw, event_id=event_dict)

    # Merge correct/error response events
    merged_events = mne.merge_events(
        events,
        [10004, 10005, 10009, 10010],
        merged_event_dict["correct_response"],
        replace_events=True,
    )
    merged_events = mne.merge_events(
        merged_events,
        [10006, 10007, 10008, 10011],
        merged_event_dict["error_response"],
        replace_events=True,
    )

    epochs = []
    bads = []
    this_reject_by_annotation = True

    # Read epochs
    epochs = mne.Epochs(
        raw=raw,
        events=merged_events,
        event_id=merged_event_dict,
        tmin=tmin,
        tmax=tmax,
        baseline=None,
        reject_by_annotation=this_reject_by_annotation,
        preload=True,
    )

    return epochs

#### Read the data

In [ ]:
df_name = "go_nogo_100_600_df_3-5_all"
pickled_data_filename = "../data/" + df_name + ".pkl"
info_filename = "../data/Demographic_Questionnaires_Behavioral_Results_N=163.csv"

# Check if data is already loaded
if os.path.isfile(pickled_data_filename):
    print("Pickled file found. Loading pickled data...")
    epochs_df = pd.read_pickle(pickled_data_filename)
    print("Done")
else:
    print("Pickled file not found. Loading data...")
    epochs_df = create_df_data(
        test_participants=False, info="all", personal=False, info_filename=info_filename
    )
    epochs_df.name = df_name
    # save loaded data into a pickle file
    epochs_df.to_pickle("../data/" + epochs_df.name + ".pkl")
    print("Done. Pickle file created")

In [ ]:
df_name = "go_nogo_500_300_df_3-5_all"
pickled_data_filename = "../data/" + df_name + ".pkl"
info_filename = "../data/Demographic_Questionnaires_Behavioral_Results_N=163.csv"

# Check if data is already loaded
if os.path.isfile(pickled_data_filename):
    print("Pickled file found. Loading pickled data...")
    epochs_df_3 = pd.read_pickle(pickled_data_filename)
    print("Done")
else:
    print("Pickled file not found. Loading data...")
    epochs_df_3 = create_df_data(
        test_participants=False, info="all", personal=False, info_filename=info_filename
    )
    epochs_df_3.name = df_name
    # save loaded data into a pickle file
    epochs_df_3.to_pickle("../data/" + epochs_df_3.name + ".pkl")
    print("Done. Pickle file created")

In [ ]:
df_name = "go_nogo_400_600_df_3-5_all"
pickled_data_filename = "../data/" + df_name + ".pkl"
info_filename = "../data/Demographic_Questionnaires_Behavioral_Results_N=163.csv"

# Check if data is already loaded
if os.path.isfile(pickled_data_filename):
    print("Pickled file found. Loading pickled data...")
    epochs_df2 = pd.read_pickle(pickled_data_filename)
    print("Done")
else:
    print("Pickled file not found. Loading data...")
    epochs_df2 = create_df_data(
        test_participants=False, info="all", personal=False, info_filename=info_filename
    )
    epochs_df2.name = df_name
    # save loaded data into a pickle file
    epochs_df2.to_pickle("../data/" + epochs_df2.name + ".pkl")
    print("Done. Pickle file created")

#### Read data for external testing

In [ ]:
df_name = "go_nogo_100_600_test_df_3-5_all"
pickled_data_filename = "../data/" + df_name + ".pkl"
info_filename = "../data/Demographic_Questionnaires_Behavioral_Results_N=163.csv"

# Check if data is already loaded
if os.path.isfile(pickled_data_filename):
    print("Pickled file found. Loading pickled data...")
    epochs_test_df = pd.read_pickle(pickled_data_filename)
    print("Done")
else:
    print("Pickled file not found. Loading data...")
    epochs_test_df = create_df_data(
        test_participants=True, info="all", personal=False, info_filename=info_filename
    )
    epochs_test_df.name = df_name
    # save loaded data into a pickle file
    epochs_test_df.to_pickle("../data/" + epochs_test_df.name + ".pkl")
    print("Done. Pickle file created")

In [ ]:
df_name = "go_nogo_500_300_test_df_3-5_all"
pickled_data_filename = "../data/" + df_name + ".pkl"
info_filename = "../data/Demographic_Questionnaires_Behavioral_Results_N=163.csv"

# Check if data is already loaded
if os.path.isfile(pickled_data_filename):
    print("Pickled file found. Loading pickled data...")
    epochs_test_df_3 = pd.read_pickle(pickled_data_filename)
    print("Done")
else:
    print("Pickled file not found. Loading data...")
    epochs_test_df_3 = create_df_data(
        test_participants=True, info="all", personal=False, info_filename=info_filename
    )
    epochs_test_df_3.name = df_name
    # save loaded data into a pickle file
    epochs_test_df_3.to_pickle("../data/" + epochs_test_df_3.name + ".pkl")
    print("Done. Pickle file created")

In [ ]:
X_train_df_100 = epochs_df
# X_train_df_400 = epochs_df2
# X_train_df_500 = epochs_df_3

---
# Between subject variation
- without spatial filter

In [ ]:
# results_df_rum = pd.read_pickle("../data/split0.3/regression_union_100-600_baselined_centered-2_diff_boxes_diff_pe-ind_diff_models.pkl")

# ern_fex = results_df_rum[results_df_rum['model'] == 'en'].best_estimator[11]['features'].transformer_list[0][1]['feature_extraction']
# pe_fex = results_df_rum[results_df_rum['model'] == 'en'].best_estimator[11]['features'].transformer_list[1][1]['feature_extraction']

In [ ]:
between_df = pd.DataFrame({'pipeline': [], 'values': []})

In [ ]:
box= [
    "Fpz", 
    "AFz",
    "Fz",
    "FCz",
    "Cz",
    "CPz",
    "Pz",
]

In [ ]:
# X_train_df_100copy = pd.DataFrame(copy.deepcopy(X_train_df_100.to_dict()))

In [ ]:
# pipeline_name = '-100:0'

# ern_features = Pipeline(steps=[
#                                 ("ern_data_extraction", ErnTransformer()),
#                                 ("ern_amplitude", ErnAmplitude2()),
#                 ])


# pe_features = Pipeline(steps = [
#                                 ("pe_data_extraction", PeTransformer(start_pe_bin=3, stop_pe_bin=8)),
#                                 ("pe_amplitude", PeAmplitude2()),
#                 ])

# ern_pe_features = FeatureUnion([("ern_features", ern_features), ("pe_features", pe_features)], n_jobs = 10)

# x_pre = Pipeline([
#             ("channels_extraction",PickChannels(channels_list=box)),
#             ("average", Evoked()),
#             ('extract_data', ExtractData()),
#             ("lowpass_filter", LowpassFilter()),
#             ("binning", BinTransformer(step=12)),
#             ("baseline", ErnBaselined()),
#             ("centering", CenteredSignalAfterBaseline3()),
#             ("features", ern_pe_features),
# ]).fit_transform(X_train_df_100copy)

# x_feature_100_between = np.sum(x_pre, axis=1)
# x_100_std_between = np.std(x_feature_100_between, axis=0)

# values = x_feature_100_between.flatten().tolist()
# names = [pipeline_name] * len(x_feature_100_between)

# temp_df = pd.DataFrame(zip(names, values), columns=['pipeline', 'values'])

# between_df = between_df.append(temp_df, ignore_index=True)

In [ ]:
# X_train_df_500copy = pd.DataFrame(copy.deepcopy(X_train_df_500.to_dict()))

In [ ]:
# pipeline_name = '-500:-300'

# ern_features = Pipeline(steps=[
#                                 ("ern_data_extraction", ErnTransformer()),
#                                 ("ern_amplitude", ErnAmplitude2()),
#                 ])


# pe_features = Pipeline(steps = [
#                                 ("pe_data_extraction", PeTransformer(start_pe_bin=3, stop_pe_bin=8)),
#                                 ("pe_amplitude", PeAmplitude2()),
#                 ])

# ern_pe_features = FeatureUnion([("ern_features", ern_features), ("pe_features", pe_features)], n_jobs = 10)

# x_pre = Pipeline([
#             ("channels_extraction",PickChannels(channels_list=box)),
#             ("average", Evoked()),
#             ('extract_data', ExtractData()),
#             ("lowpass_filter", LowpassFilter()),
#             ("binning", BinTransformer(step=12)),
#             ("baseline", ErnBaselined()),
#             ("centering", CenteredSignalAfterBaseline3()),
#             ("features", ern_pe_features),
# ]).fit_transform(X_train_df_500copy)

# x_feature_500_between = np.sum(x_pre, axis=1)
# x_500_std_between = np.std(x_feature_500_between, axis=0)

# values = x_feature_500_between.flatten().tolist()
# names = [pipeline_name] * len(x_feature_500_between)

# temp_df = pd.DataFrame(zip(names, values), columns=['pipeline', 'values'])

# between_df = between_df.append(temp_df, ignore_index=True)

- different lowpass filters

In [ ]:
for cutoff in [40]:
    X_train_df_100copy = pd.DataFrame(copy.deepcopy(X_train_df_100.to_dict()))
    
    pipeline_name = '-100:0 '+ str(cutoff)

    ern_features = Pipeline(steps=[
                                    ("ern_data_extraction", ErnTransformer()),
                                    ("ern_amplitude", ErnAmplitude2_prim()),
                    ])


    pe_features = Pipeline(steps = [
                                    ("pe_data_extraction", PeTransformer(start_pe_bin=2, stop_pe_bin=7)),
                                    ("pe_amplitude", PeAmplitude2()),
                    ])

    ern_pe_features = FeatureUnion([("ern_features", ern_features), ("pe_features", pe_features)], n_jobs = 10)

    x_pre = Pipeline([
                ("channels_extraction",PickChannels(channels_list=box)),
                ("average", Evoked()),
                ('extract_data', ExtractData()),
                ("lowpass_filter", LowpassFilter(cutoff=cutoff)),
                ('neg', ReverseSignal()),
                ("binning", BinTransformer(step=12)),
                ("baseline", ErnBaselined()),
                ("centering", CenteredSignalAfterBaseline3()),
                ("features", ern_pe_features),
    ]).fit_transform(X_train_df_100copy)

    x_feature_100_between = np.sum(x_pre, axis=1)
    x_100_std_between = np.std(x_feature_100_between, axis=0)

    values = x_feature_100_between.flatten().tolist()
    names = [pipeline_name] * len(x_feature_100_between)

    temp_df = pd.DataFrame(zip(names, values), columns=['pipeline', 'values'])

    between_df = between_df.append(temp_df, ignore_index=True)
    
    ###########################################################################################
   

In [ ]:
for cutoff in [15,20,30,40]:
    X_train_df_100copy = pd.DataFrame(copy.deepcopy(X_train_df_100.to_dict()))
    
    pipeline_name = '-100:0 '+ str(cutoff) + ' no BS'

    ern_features = Pipeline(steps=[
                                    ("ern_data_extraction", ErnTransformer()),
                                    ("ern_amplitude", ErnAmplitude2()),
                    ])


    pe_features = Pipeline(steps = [
                                    ("pe_data_extraction", PeTransformer(start_pe_bin=3, stop_pe_bin=8)),
                                    ("pe_amplitude", PeAmplitude2()),
                    ])

    ern_pe_features = FeatureUnion([("ern_features", ern_features), ("pe_features", pe_features)], n_jobs = 10)

    x_pre = Pipeline([
                ("channels_extraction",PickChannels(channels_list=box)),
                ("average", Evoked()),
                ('extract_data', ExtractData()),
                ("lowpass_filter", LowpassFilter(cutoff=cutoff)),
                ("binning", BinTransformer(step=12)),
                # ("baseline", ErnBaselined()),
                ("centering", CenteredSignalAfterBaseline3()),
                ("features", ern_pe_features),
    ]).fit_transform(X_train_df_100copy)

    x_feature_100_between = np.sum(x_pre, axis=1)
    x_100_std_between = np.std(x_feature_100_between, axis=0)

    values = x_feature_100_between.flatten().tolist()
    names = [pipeline_name] * len(x_feature_100_between)

    temp_df = pd.DataFrame(zip(names, values), columns=['pipeline', 'values'])

    between_df = between_df.append(temp_df, ignore_index=True)
    

In [ ]:
for cutoff in [40]:
    X_train_df_500copy = pd.DataFrame(copy.deepcopy(X_train_df_500.to_dict()))
    
    pipeline_name = '-500:-300 '+ str(cutoff)

    ern_features = Pipeline(steps=[
                                    ("ern_data_extraction", ErnTransformer()),
                                    ("ern_amplitude", ErnAmplitude2()),
                    ])


    pe_features = Pipeline(steps = [
                                    ("pe_data_extraction", PeTransformer(start_pe_bin=3, stop_pe_bin=8)),
                                    ("pe_amplitude", PeAmplitude2()),
                    ])

    ern_pe_features = FeatureUnion([("ern_features", ern_features), ("pe_features", pe_features)], n_jobs = 10)

    x_pre = Pipeline([
                ("channels_extraction",PickChannels(channels_list=box)),
                ("average", Evoked()),
                ('extract_data', ExtractData()),
                ("lowpass_filter", LowpassFilter(cutoff=cutoff)),
                ("binning", BinTransformer(step=12)),
                ("baseline", ErnBaselined()),
                ("centering", CenteredSignalAfterBaseline3()),
                ("features", ern_pe_features),
    ]).fit_transform(X_train_df_500copy)

    x_feature_500_between = np.sum(x_pre, axis=1)
    x_500_std_between = np.std(x_feature_500_between, axis=0)

    values = x_feature_500_between.flatten().tolist()
    names = [pipeline_name] * len(x_feature_500_between)

    temp_df = pd.DataFrame(zip(names, values), columns=['pipeline', 'values'])

    between_df = between_df.append(temp_df, ignore_index=True)
    
    ##########################################################################
    
    X_train_df_500copy = pd.DataFrame(copy.deepcopy(X_train_df_500.to_dict()))
    
    pipeline_name = '-500:-300 '+ str(cutoff) + ' no BS'

    ern_features = Pipeline(steps=[
                                    ("ern_data_extraction", ErnTransformer()),
                                    ("ern_amplitude", ErnAmplitude2()),
                    ])


    pe_features = Pipeline(steps = [
                                    ("pe_data_extraction", PeTransformer(start_pe_bin=3, stop_pe_bin=8)),
                                    ("pe_amplitude", PeAmplitude2()),
                    ])

    ern_pe_features = FeatureUnion([("ern_features", ern_features), ("pe_features", pe_features)], n_jobs = 10)

    x_pre = Pipeline([
                ("channels_extraction",PickChannels(channels_list=box)),
                ("average", Evoked()),
                ('extract_data', ExtractData()),
                ("lowpass_filter", LowpassFilter(cutoff=cutoff)),
                ("binning", BinTransformer(step=12)),
                # ("baseline", ErnBaselined()),
                ("centering", CenteredSignalAfterBaseline3()),
                ("features", ern_pe_features),
    ]).fit_transform(X_train_df_500copy)

    x_feature_500_between = np.sum(x_pre, axis=1)
    x_500_std_between = np.std(x_feature_500_between, axis=0)

    values = x_feature_500_between.flatten().tolist()
    names = [pipeline_name] * len(x_feature_500_between)

    temp_df = pd.DataFrame(zip(names, values), columns=['pipeline', 'values'])

    between_df = between_df.append(temp_df, ignore_index=True)

- with spatial filter

In [ ]:
# X_train_df_100copy = pd.DataFrame(copy.deepcopy(X_train_df_100.to_dict()))

In [ ]:
for cutoff in [40]:
    X_train_df_100copy = pd.DataFrame(copy.deepcopy(X_train_df_100.to_dict()))

    pipeline_name = '-100:0 SF '+ str(cutoff)

    ern_features = Pipeline(steps=[
                                    ("ern_data_extraction", ErnTransformer()),
                                    ("ern_amplitude", ErnAmplitude2_prim()),
                                    # ("data_channel_swap", ChannelDataSwap()),
                                    # ("postprocessing", PostprocessingTransformer()),
                                    # ("scaler", StandardScaler()),
                    ])


    pe_features = Pipeline(steps = [
                                    ("pe_data_extraction", PeTransformer(start_pe_bin=2, stop_pe_bin=8)),
                                    ("pe_amplitude", PeAmplitude2()),
                                    # ("data_channel_swap", ChannelDataSwap()),
                                    # ("postprocessing", PostprocessingTransformer()),
                                    # ("scaler", StandardScaler()),
                    ])

    ern_pe_features = FeatureUnion([("ern_features", ern_features), ("pe_features", pe_features)], n_jobs = 10)


    x_pre = Pipeline([
                ("channels_extraction",PickChannels(channels_list=box)),
                ("average", Evoked()),
                ('extract_data', ExtractData()),
                ("spatial_filter_preprocessing", SpatialFilterPreprocessing()),
                ("spatial_filter",PCA(n_components=3, random_state=random_state)),
                ("spatial_filter_postprocessing",SpatialFilterPostprocessing(timepoints_count=181)),
                ("lowpass_filter", LowpassFilter(cutoff=cutoff)),
                ('neg', ReverseComponent3()),
                ("binning", BinTransformer(step=12)),
                ("baseline", ErnBaselined()),
                ("centering", CenteredSignalAfterBaseline3()),
                ("features", ern_pe_features),
                # ("ern_amplitude", ErnAmplitude2())
    ]).fit_transform(X_train_df_100copy)

    x_feature_100_sf_between = np.sum(x_pre, axis=1)
    x_100_std_sf_between = np.std(x_feature_100_sf_between, axis=0)

    values = x_feature_100_sf_between.flatten().tolist()
    names = [pipeline_name] * len(x_feature_100_sf_between)

    temp_df = pd.DataFrame(zip(names, values), columns=['pipeline', 'values'])

    between_df = between_df.append(temp_df, ignore_index=True)
    
    #####################################################################
    
    X_train_df_100copy = pd.DataFrame(copy.deepcopy(X_train_df_100.to_dict()))

    pipeline_name = '-100:0 SF '+ str(cutoff) + ' no BS'

    ern_features = Pipeline(steps=[
                                    ("ern_data_extraction", ErnTransformer()),
                                    ("ern_amplitude", ErnAmplitude2()),
                                    # ("data_channel_swap", ChannelDataSwap()),
                                    # ("postprocessing", PostprocessingTransformer()),
                                    # ("scaler", StandardScaler()),
                    ])


    pe_features = Pipeline(steps = [
                                    ("pe_data_extraction", PeTransformer(start_pe_bin=2, stop_pe_bin=8)),
                                    ("pe_amplitude", PeAmplitude2()),
                                    # ("data_channel_swap", ChannelDataSwap()),
                                    # ("postprocessing", PostprocessingTransformer()),
                                    # ("scaler", StandardScaler()),
                    ])

    ern_pe_features = FeatureUnion([("ern_features", ern_features), ("pe_features", pe_features)], n_jobs = 10)


    x_pre = Pipeline([
                ("channels_extraction",PickChannels(channels_list=box)),
                ("average", Evoked()),
                ('extract_data', ExtractData()),
                ("spatial_filter_preprocessing", SpatialFilterPreprocessing()),
                ("spatial_filter",PCA(n_components=3, random_state=random_state)),
                ("spatial_filter_postprocessing",SpatialFilterPostprocessing(timepoints_count=181)),
                ("lowpass_filter", LowpassFilter(cutoff=cutoff)),
                ('neg', ReverseComponent3()),
                ("binning", BinTransformer(step=12)),
                # ("baseline", ErnBaselined()),
                ("centering", CenteredSignalAfterBaseline3()),
                ("features", ern_pe_features),
                # ("ern_amplitude", ErnAmplitude2())
    ]).fit_transform(X_train_df_100copy)

    x_feature_100_sf_between = np.sum(x_pre, axis=1)
    x_100_std_sf_between = np.std(x_feature_100_sf_between, axis=0)

    values = x_feature_100_sf_between.flatten().tolist()
    names = [pipeline_name] * len(x_feature_100_sf_between)

    temp_df = pd.DataFrame(zip(names, values), columns=['pipeline', 'values'])

    between_df = between_df.append(temp_df, ignore_index=True)

In [ ]:
# X_train_df_500copy = pd.DataFrame(copy.deepcopy(X_train_df_500.to_dict()))

In [ ]:
for cutoff in [15,20,30,40]:
    X_train_df_500copy = pd.DataFrame(copy.deepcopy(X_train_df_500.to_dict()))
    
    pipeline_name = '-500:-300 SF ' + str(cutoff)

    ern_features = Pipeline(steps=[
                                    ("ern_data_extraction", ErnTransformer()),
                                    ("ern_amplitude", ErnAmplitude2()),
                    ])


    pe_features = Pipeline(steps = [
                                    ("pe_data_extraction", PeTransformer(start_pe_bin=2, stop_pe_bin=8)),
                                    ("pe_amplitude", PeAmplitude2()),
                    ])

    ern_pe_features = FeatureUnion([("ern_features", ern_features), ("pe_features", pe_features)], n_jobs = 10)


    x_pre = Pipeline([
                ("channels_extraction",PickChannels(channels_list=box)),
                ("average", Evoked()),
                ('extract_data', ExtractData()),
                ("spatial_filter_preprocessing", SpatialFilterPreprocessing()),
                ("spatial_filter",PCA(n_components=3, random_state=random_state)),
                ("spatial_filter_postprocessing",SpatialFilterPostprocessing(timepoints_count=181)),
                ("lowpass_filter", LowpassFilter(cutoff=cutoff)),
                ('neg', ReverseComponent3()),
                ("binning", BinTransformer(step=12)),
                ("baseline", ErnBaselined()),
                ("centering", CenteredSignalAfterBaseline3()),
                ("features", ern_pe_features),
    ]).fit_transform(X_train_df_500copy)

    x_feature_500_sf_between = np.sum(x_pre, axis=1)
    x_500_std_sf_between = np.std(x_feature_500_sf_between, axis=0)

    values = x_feature_500_sf_between.flatten().tolist()
    names = [pipeline_name] * len(x_feature_500_sf_between)

    temp_df = pd.DataFrame(zip(names, values), columns=['pipeline', 'values'])

    between_df = between_df.append(temp_df, ignore_index=True)
    
    ##################################################################
    X_train_df_500copy = pd.DataFrame(copy.deepcopy(X_train_df_500.to_dict()))
    
    pipeline_name = '-500:-300 SF ' + str(cutoff) + ' no BS'

    ern_features = Pipeline(steps=[
                                    ("ern_data_extraction", ErnTransformer()),
                                    ("ern_amplitude", ErnAmplitude2()),
                    ])


    pe_features = Pipeline(steps = [
                                    ("pe_data_extraction", PeTransformer(start_pe_bin=2, stop_pe_bin=8)),
                                    ("pe_amplitude", PeAmplitude2()),
                    ])

    ern_pe_features = FeatureUnion([("ern_features", ern_features), ("pe_features", pe_features)], n_jobs = 10)


    x_pre = Pipeline([
                ("channels_extraction",PickChannels(channels_list=box)),
                ("average", Evoked()),
                ('extract_data', ExtractData()),
                ("spatial_filter_preprocessing", SpatialFilterPreprocessing()),
                ("spatial_filter",PCA(n_components=3, random_state=random_state)),
                ("spatial_filter_postprocessing",SpatialFilterPostprocessing(timepoints_count=181)),
                ("lowpass_filter", LowpassFilter(cutoff=cutoff)),
                ('neg', ReverseComponent3()),
                ("binning", BinTransformer(step=12)),
                # ("baseline", ErnBaselined()),
                ("centering", CenteredSignalAfterBaseline3()),
                ("features", ern_pe_features),
    ]).fit_transform(X_train_df_500copy)

    x_feature_500_sf_between = np.sum(x_pre, axis=1)
    x_500_std_sf_between = np.std(x_feature_500_sf_between, axis=0)

    values = x_feature_500_sf_between.flatten().tolist()
    names = [pipeline_name] * len(x_feature_500_sf_between)

    temp_df = pd.DataFrame(zip(names, values), columns=['pipeline', 'values'])

    between_df = between_df.append(temp_df, ignore_index=True)

- with spatial filter and feature extraction

In [ ]:
# X_train_df_100copy = pd.DataFrame(copy.deepcopy(X_train_df_100.to_dict()))

In [ ]:
# pipeline_name = '-100:0 SF EX'

# x_pre = Pipeline([
#             ("channels_extraction",PickChannels(channels_list=box)),
#             ("average", Evoked()),
#             ('extract_data', ExtractData()),
#             ("spatial_filter_preprocessing", SpatialFilterPreprocessing()),
#             ("spatial_filter",PCA(n_components=3, random_state=random_state)),
#             ("spatial_filter_postprocessing",SpatialFilterPostprocessing(timepoints_count=181)),
#             ("lowpass_filter", LowpassFilter()),
#             ('neg', ReverseComponent3()),
#             ("binning", BinTransformer(step=12)),
#             ("baseline", ErnBaselined()),
#             ("centering", CenteredSignalAfterBaseline3()),
#             # ("features", ern_pe_features),
#             # ("ern_amplitude", ErnAmplitude2())
# ]).fit_transform(X_train_df_100copy)


# ern_features_pre = Pipeline(steps=[
#                                 ("ern_data_extraction", ErnTransformer()),
#                                 ("ern_amplitude", ErnAmplitude2()),
#                                 ("data_channel_swap", ChannelDataSwap()),
#                                 ("postprocessing", PostprocessingTransformer()),
#                                 # ("scaler", StandardScaler()),
#                                 # ("feature_extraction", FastICA(random_state=random_state, n_components=3))
#                 ]).fit_transform(x_pre)


# ern_features = Pipeline([("feature_extraction", ern_fex)]).transform(ern_features_pre)


# pe_features_pre = Pipeline(steps = [
#                                 ("pe_data_extraction", PeTransformer(start_pe_bin=2, stop_pe_bin=8)),
#                                 ("pe_amplitude", PeAmplitude2()),
#                                 ("data_channel_swap", ChannelDataSwap()),
#                                 ("postprocessing", PostprocessingTransformer()),
#                                 # ("scaler", StandardScaler()),
#                                 # ("feature_extraction", FastICA(random_state=random_state, n_components=3))

#                 ]).fit_transform(x_pre)

# pe_features = Pipeline([("feature_extraction", pe_fex)]).transform(pe_features_pre)

# # ern_pe_features = FeatureUnion([("ern_features", ern_features), ("pe_features", pe_features)], n_jobs = 10).fit_transform()

# x_pre2 = zip(ern_features, pe_features)
# x_pre2 = np.array(list(x_pre2)).reshape(x_pre.shape[0],-1)
        
# x_feature_100_sf_between = np.sum(x_pre2, axis=1)
# x_100_std_sf_between = np.std(x_feature_100_sf_between, axis=0)

# values = x_feature_100_sf_between.flatten().tolist()
# names = [pipeline_name] * len(x_feature_100_sf_between)

# temp_df = pd.DataFrame(zip(names, values), columns=['pipeline', 'values'])

# between_df = between_df.append(temp_df, ignore_index=True)

In [ ]:
# X_train_df_500copy = pd.DataFrame(copy.deepcopy(X_train_df_500.to_dict()))

In [ ]:
# pipeline_name = '-500:-300 SF EX'

# # ern_features = Pipeline(steps=[
# #                                 ("ern_data_extraction", ErnTransformer()),
# #                                 ("ern_amplitude", ErnAmplitude2()),
# #                                 ("data_channel_swap", ChannelDataSwap()),
# #                                 ("postprocessing", PostprocessingTransformer()),
# #                                 ("scaler", StandardScaler()),
# #                                 ("feature_extraction", FastICA(random_state=random_state, n_components=3))
# #                 ])


# # pe_features = Pipeline(steps = [
# #                                 ("pe_data_extraction", PeTransformer(start_pe_bin=2, stop_pe_bin=8)),
# #                                 ("pe_amplitude", PeAmplitude2()),
# #                                 ("data_channel_swap", ChannelDataSwap()),
# #                                 ("postprocessing", PostprocessingTransformer()),
# #                                 ("scaler", StandardScaler()),
# #                                 ("feature_extraction", FastICA(random_state=random_state, n_components=3))
# #                 ])

# # ern_pe_features = FeatureUnion([("ern_features", ern_features), ("pe_features", pe_features)], n_jobs = 10)


# # x_pre = Pipeline([
# #             ("channels_extraction",PickChannels(channels_list=box)),
# #             ("average", Evoked()),
# #             ('extract_data', ExtractData()),
# #             ("spatial_filter_preprocessing", SpatialFilterPreprocessing()),
# #             ("spatial_filter",PCA(n_components=3, random_state=random_state)),
# #             ("spatial_filter_postprocessing",SpatialFilterPostprocessing(timepoints_count=181)),
# #             ("lowpass_filter", LowpassFilter()),
# #             ('neg', ReverseComponent3()),
# #             ("binning", BinTransformer(step=12)),
# #             ("baseline", ErnBaselined()),
# #             ("centering", CenteredSignalAfterBaseline3()),
# #             ("features", ern_pe_features),
# #             # ("ern_amplitude", ErnAmplitude2())
# # ]).fit_transform(X_train_df_500copy)

# x_pre = Pipeline([
#             ("channels_extraction",PickChannels(channels_list=box)),
#             ("average", Evoked()),
#             ('extract_data', ExtractData()),
#             ("spatial_filter_preprocessing", SpatialFilterPreprocessing()),
#             ("spatial_filter",PCA(n_components=3, random_state=random_state)),
#             ("spatial_filter_postprocessing",SpatialFilterPostprocessing(timepoints_count=181)),
#             ("lowpass_filter", LowpassFilter()),
#             ('neg', ReverseComponent3()),
#             ("binning", BinTransformer(step=12)),
#             ("baseline", ErnBaselined()),
#             ("centering", CenteredSignalAfterBaseline3()),
#             # ("features", ern_pe_features),
#             # ("ern_amplitude", ErnAmplitude2())
# ]).fit_transform(X_train_df_500copy)


# ern_features_pre = Pipeline(steps=[
#                                 ("ern_data_extraction", ErnTransformer()),
#                                 ("ern_amplitude", ErnAmplitude2()),
#                                 ("data_channel_swap", ChannelDataSwap()),
#                                 ("postprocessing", PostprocessingTransformer()),
#                                 # ("scaler", StandardScaler()),
#                                 # ("feature_extraction", FastICA(random_state=random_state, n_components=3))
#                 ]).fit_transform(x_pre)


# ern_features = Pipeline([("feature_extraction", ern_fex)]).transform(ern_features_pre)


# pe_features_pre = Pipeline(steps = [
#                                 ("pe_data_extraction", PeTransformer(start_pe_bin=2, stop_pe_bin=8)),
#                                 ("pe_amplitude", PeAmplitude2()),
#                                 ("data_channel_swap", ChannelDataSwap()),
#                                 ("postprocessing", PostprocessingTransformer()),
#                                 # ("scaler", StandardScaler()),
#                                 # ("feature_extraction", FastICA(random_state=random_state, n_components=3))

#                 ]).fit_transform(x_pre)

# pe_features = Pipeline([("feature_extraction", pe_fex)]).transform(pe_features_pre)

# # ern_pe_features = FeatureUnion([("ern_features", ern_features), ("pe_features", pe_features)], n_jobs = 10).fit_transform()

# x_pre2 = zip(ern_features, pe_features)
# x_pre2 = np.array(list(x_pre2)).reshape(x_pre.shape[0],-1)

# x_feature_500_sf_between = np.sum(x_pre2, axis=1)
# x_500_std_sf_between = np.std(x_feature_500_sf_between, axis=0)

# values = x_feature_500_sf_between.flatten().tolist()
# names = [pipeline_name] * len(x_feature_500_sf_between)

# temp_df = pd.DataFrame(zip(names, values), columns=['pipeline', 'values'])

# between_df = between_df.append(temp_df, ignore_index=True)

### Visualization

In [ ]:
import seaborn as sns
sns.set(rc={'figure.figsize':(10,8)},font_scale = 1.2)
sns.set_style("whitegrid")

ax = sns.pointplot(x = 'values', y='pipeline', data = between_df, orient='h', join=False, estimator=np.std, ci=95,capsize=.05,)
# ax.figure.savefig("between_subject_std_ern_pe_lowpass.png")

In [ ]:
import seaborn as sns
sns.set(rc={'figure.figsize':(10,8)},font_scale = 1.2)
sns.set_style("whitegrid")

ax = sns.pointplot(x = 'values', y='pipeline', data = between_df, orient='h', join=False, estimator=np.std, ci=95,capsize=.05,)
# ax.figure.savefig("between_subject_std_ern_pe_lowpass.png")

----
# WITHIN SUBJECT
- without spatial filter

In [ ]:
within_df = pd.DataFrame({'pipeline': [], 'values': []})

## -100 to 0

In [ ]:
# X_train_df_100copy = pd.DataFrame(copy.deepcopy(X_train_df_100.to_dict()))

In [ ]:
for cutoff in [40]:
    X_train_df_100copy = pd.DataFrame(copy.deepcopy(X_train_df_100.to_dict()))

    pipeline_name = '-100:0 ' + str(cutoff)

    for i in range(0,len(X_train_df_100copy)):

        X = X_train_df_100copy[i:i+1]    
        x_pre = Pipeline([
                ("channels_extraction",PickChannels(channels_list=box)),
                ('extract_data', ExtractDataEpochs()),
        ]).fit_transform(X)  

        x_pre = x_pre[0] 

        ern_features = Pipeline(steps=[
                                    ("ern_data_extraction", ErnTransformer()),
                                    ("ern_amplitude", ErnAmplitude2_prim()),
                                    # ("data_channel_swap", ChannelDataSwap()),
                                    # ("postprocessing", PostprocessingTransformer()),
                                    # ("scaler", StandardScaler()),
                                    # ("feature_extraction", FastICA(random_state=random_state, n_components=3))
                    ])


        pe_features = Pipeline(steps = [
                                    ("pe_data_extraction", PeTransformer(start_pe_bin=2, stop_pe_bin=7)),
                                    ("pe_amplitude", PeAmplitude2()),
                                    # ("data_channel_swap", ChannelDataSwap()),
                                    # ("postprocessing", PostprocessingTransformer()),
                                    # ("scaler", StandardScaler()),
                                    # ("feature_extraction", FastICA(random_state=random_state, n_components=3))
                    ])

        ern_pe_features = FeatureUnion([("ern_features", ern_features), ("pe_features", pe_features)], n_jobs = 10)


        x_pre2 = Pipeline([
            ("lowpass_filter", LowpassFilter(cutoff=cutoff)),
            ('neg', ReverseSignal()),
            ("binning", BinTransformer(step=12)),
            ("baseline", ErnBaselined()),
            ("centering", CenteredSignalAfterBaseline3()),
            ("features", ern_pe_features),
            # ("ern_amplitude", ErnAmplitude2()),
        ]).fit_transform(x_pre)

        # f_vector = np.mean(x_pre2, axis=1)
        f_vector = np.sum(x_pre2, axis=1)
        f_variance = np.std(f_vector)

        data = {'pipeline' : pipeline_name,
                'values' : f_variance,
               }

        within_df = within_df.append(data, ignore_index = True)
        
        ##########################################################################
        
        

In [ ]:
for cutoff in [15,20,30,40]:
    X_train_df_100copy = pd.DataFrame(copy.deepcopy(X_train_df_100.to_dict()))

    pipeline_name = '-100:0 ' + str(cutoff) + ' no BS'

    for i in range(0,len(X_train_df_100copy)):

        X = X_train_df_100copy[i:i+1]    
        x_pre = Pipeline([
                ("channels_extraction",PickChannels(channels_list=box)),
                ('extract_data', ExtractDataEpochs()),
        ]).fit_transform(X)  

        x_pre = x_pre[0] 

        ern_features = Pipeline(steps=[
                                    ("ern_data_extraction", ErnTransformer()),
                                    ("ern_amplitude", ErnAmplitude2()),
                                    # ("data_channel_swap", ChannelDataSwap()),
                                    # ("postprocessing", PostprocessingTransformer()),
                                    # ("scaler", StandardScaler()),
                                    # ("feature_extraction", FastICA(random_state=random_state, n_components=3))
                    ])


        pe_features = Pipeline(steps = [
                                    ("pe_data_extraction", PeTransformer(start_pe_bin=3, stop_pe_bin=8)),
                                    ("pe_amplitude", PeAmplitude2()),
                                    # ("data_channel_swap", ChannelDataSwap()),
                                    # ("postprocessing", PostprocessingTransformer()),
                                    # ("scaler", StandardScaler()),
                                    # ("feature_extraction", FastICA(random_state=random_state, n_components=3))
                    ])

        ern_pe_features = FeatureUnion([("ern_features", ern_features), ("pe_features", pe_features)], n_jobs = 10)


        x_pre2 = Pipeline([
            ("lowpass_filter", LowpassFilter(cutoff=cutoff)),
            ("binning", BinTransformer(step=12)),
            # ("baseline", ErnBaselined()),
            ("centering", CenteredSignalAfterBaseline3()),
            ("features", ern_pe_features),
            # ("ern_amplitude", ErnAmplitude2()),
        ]).fit_transform(x_pre)

        # f_vector = np.mean(x_pre2, axis=1)
        f_vector = np.sum(x_pre2, axis=1)
        f_variance = np.std(f_vector)

        data = {'pipeline' : pipeline_name,
                'values' : f_variance,
               }

        within_df = within_df.append(data, ignore_index = True)

## -500 to -300

In [ ]:
# X_train_df_500copy = pd.DataFrame(copy.deepcopy(X_train_df_500.to_dict()))

In [ ]:
for cutoff in [15,20,30,40]:
    X_train_df_500copy = pd.DataFrame(copy.deepcopy(X_train_df_500.to_dict()))

    pipeline_name = '-500:-300 ' + str(cutoff)

    for i in range(0,len(X_train_df_500copy)):

        X = X_train_df_500copy[i:i+1]    
        x_pre = Pipeline([
                ("channels_extraction",PickChannels(channels_list=box)),
                ('extract_data', ExtractDataEpochs()),
        ]).fit_transform(X)  
        x_pre = x_pre[0] 

        ern_features = Pipeline(steps=[
                                    ("ern_data_extraction", ErnTransformer()),
                                    ("ern_amplitude", ErnAmplitude2()),
                                    # ("data_channel_swap", ChannelDataSwap()),
                                    # ("postprocessing", PostprocessingTransformer()),
                                    # ("scaler", StandardScaler()),
                                    # ("feature_extraction", FastICA(random_state=random_state, n_components=3))
                    ])


        pe_features = Pipeline(steps = [
                                    ("pe_data_extraction", PeTransformer(start_pe_bin=3, stop_pe_bin=8)),
                                    ("pe_amplitude", PeAmplitude2()),
                                    # ("data_channel_swap", ChannelDataSwap()),
                                    # ("postprocessing", PostprocessingTransformer()),
                                    # ("scaler", StandardScaler()),
                                    # ("feature_extraction", FastICA(random_state=random_state, n_components=3))
                    ])

        ern_pe_features = FeatureUnion([("ern_features", ern_features), ("pe_features", pe_features)], n_jobs = 10)


        x_pre2 = Pipeline([
            ("lowpass_filter", LowpassFilter(cutoff=cutoff)),
            ("binning", BinTransformer(step=12)),
            ("baseline", ErnBaselined()),
            ("centering", CenteredSignalAfterBaseline3()),
            ("features", ern_pe_features),
            # ("ern_amplitude", ErnAmplitude2()),
        ]).fit_transform(x_pre)

        # f_vector = np.mean(x_pre2, axis=1)
        f_vector = np.sum(x_pre2, axis=1)

        f_variance = np.std(f_vector)

        data = {'pipeline' : pipeline_name,
                'values' : f_variance,
               }

        within_df = within_df.append(data, ignore_index = True)
        
        #############################################################################
    X_train_df_500copy = pd.DataFrame(copy.deepcopy(X_train_df_500.to_dict()))

    pipeline_name = '-500:-300 ' + str(cutoff)  + ' no BS'

    for i in range(0,len(X_train_df_500copy)):

        X = X_train_df_500copy[i:i+1]    
        x_pre = Pipeline([
                ("channels_extraction",PickChannels(channels_list=box)),
                ('extract_data', ExtractDataEpochs()),
        ]).fit_transform(X)  
        x_pre = x_pre[0] 

        ern_features = Pipeline(steps=[
                                    ("ern_data_extraction", ErnTransformer()),
                                    ("ern_amplitude", ErnAmplitude2()),
                                    # ("data_channel_swap", ChannelDataSwap()),
                                    # ("postprocessing", PostprocessingTransformer()),
                                    # ("scaler", StandardScaler()),
                                    # ("feature_extraction", FastICA(random_state=random_state, n_components=3))
                    ])


        pe_features = Pipeline(steps = [
                                    ("pe_data_extraction", PeTransformer(start_pe_bin=3, stop_pe_bin=8)),
                                    ("pe_amplitude", PeAmplitude2()),
                                    # ("data_channel_swap", ChannelDataSwap()),
                                    # ("postprocessing", PostprocessingTransformer()),
                                    # ("scaler", StandardScaler()),
                                    # ("feature_extraction", FastICA(random_state=random_state, n_components=3))
                    ])

        ern_pe_features = FeatureUnion([("ern_features", ern_features), ("pe_features", pe_features)], n_jobs = 10)


        x_pre2 = Pipeline([
            ("lowpass_filter", LowpassFilter(cutoff=cutoff)),
            ("binning", BinTransformer(step=12)),
            # ("baseline", ErnBaselined()),
            ("centering", CenteredSignalAfterBaseline3()),
            ("features", ern_pe_features),
            # ("ern_amplitude", ErnAmplitude2()),
        ]).fit_transform(x_pre)

        # f_vector = np.mean(x_pre2, axis=1)
        f_vector = np.sum(x_pre2, axis=1)

        f_variance = np.std(f_vector)

        data = {'pipeline' : pipeline_name,
                'values' : f_variance,
               }

        within_df = within_df.append(data, ignore_index = True)

- With spatial filter

### -100 to 0

In [ ]:
X_train_df_100copy = pd.DataFrame(copy.deepcopy(X_train_df_100.to_dict()))

In [ ]:
pre_pip_for_spatial_filter = Pipeline([
        ("channels_extraction",PickChannels(channels_list = box)),
        ("average", Evoked()),
        ('extract_averaged_data', ExtractData()),
        # ("narrow_indices", NarrowIndices(start=76, stop=257)),
        ("spatial_filter_preprocessing", SpatialFilterPreprocessing()),
        ("spatial_filter",PCA(n_components=3, random_state=random_state)),
]).fit(X_train_df_100copy)

spatial_filter = pre_pip_for_spatial_filter['spatial_filter']

In [ ]:
# X_train_df_100copy = pd.DataFrame(copy.deepcopy(X_train_df_100.to_dict()))

In [ ]:
for cutoff in [40]:
    X_train_df_100copy = pd.DataFrame(copy.deepcopy(X_train_df_100.to_dict()))

    pipeline_name = '-100:0 SF ' + str(cutoff)

    for i in range(0,len(X_train_df_100copy)):

        X = X_train_df_100copy[i:i+1]    
        x_pre = Pipeline([
                ("channels_extraction",PickChannels(channels_list=box)),
                ('extract_data', ExtractDataEpochs()),
        ]).fit_transform(X)  

        x_pre = x_pre[0] 

        x_pre_pre = Pipeline([
            ("spatial_filter_preprocessing", SpatialFilterPreprocessing()),
            ("spatial_filter", spatial_filter),
            ("spatial_filter_postprocessing",SpatialFilterPostprocessing(timepoints_count=181)),
            ("lowpass_filter", LowpassFilter(cutoff=cutoff)),
            ('neg', ReverseComponent3()),
            ("binning", BinTransformer(step=12)),
            ("baseline", ErnBaselined()),
            ("centering", CenteredSignalAfterBaseline3())]).transform(x_pre)


        ern_features_pre = Pipeline(steps=[
                                    ("ern_data_extraction", ErnTransformer()),
                                    ("ern_amplitude", ErnAmplitude2_prim()),
                                    # ("data_channel_swap", ChannelDataSwap()),
                                    # ("postprocessing", PostprocessingTransformer()),
                                    # ("scaler", StandardScaler()),
                    ])

        # ern_features = Pipeline([("feature_extraction", ern_fex)]).transform(ern_features_pre)


        pe_features_pre = Pipeline(steps = [
                                    ("pe_data_extraction", PeTransformer(start_pe_bin=3, stop_pe_bin=8)),
                                    ("pe_amplitude", PeAmplitude2()),
                                    # ("data_channel_swap", ChannelDataSwap()),
                                    # ("postprocessing", PostprocessingTransformer()),
                                    # ("scaler", StandardScaler()),
                    ])

        # pe_features = Pipeline([("feature_extraction", pe_fex)]).transform(pe_features_pre)

    #     x_pre2 = zip(ern_features, pe_features)
    #     x_pre2 = np.array(list(x_pre2)).reshape(x_pre.shape[0],-1)

    #     print(x_pre2.shape)

        ern_pe_features = FeatureUnion([("ern_features", ern_features_pre), ("pe_features", pe_features_pre)], n_jobs = 10)

        x_pre2 = Pipeline([
            ("features", ern_pe_features),
        ]).fit_transform(x_pre_pre)

        # f_vector = np.mean(x_pre2, axis=1)
        f_vector = np.sum(x_pre2, axis=1)

        f_variance = np.std(f_vector)

        data = {'pipeline' : pipeline_name,
                'values' : f_variance,
               }

        within_df = within_df.append(data, ignore_index = True)
        
        ########################################################################
    X_train_df_100copy = pd.DataFrame(copy.deepcopy(X_train_df_100.to_dict()))

    pipeline_name = '-100:0 SF ' + str(cutoff)  + ' no BS'


    for i in range(0,len(X_train_df_100copy)):

        X = X_train_df_100copy[i:i+1]    
        x_pre = Pipeline([
                ("channels_extraction",PickChannels(channels_list=box)),
                ('extract_data', ExtractDataEpochs()),
        ]).fit_transform(X)  

        x_pre = x_pre[0] 

        x_pre_pre = Pipeline([
            ("spatial_filter_preprocessing", SpatialFilterPreprocessing()),
            ("spatial_filter", spatial_filter),
            ("spatial_filter_postprocessing",SpatialFilterPostprocessing(timepoints_count=181)),
            ("lowpass_filter", LowpassFilter(cutoff=cutoff)),
            ('neg', ReverseComponent3()),
            ("binning", BinTransformer(step=12)),
            # ("baseline", ErnBaselined()),
            ("centering", CenteredSignalAfterBaseline3())]).transform(x_pre)


        ern_features_pre = Pipeline(steps=[
                                    ("ern_data_extraction", ErnTransformer()),
                                    ("ern_amplitude", ErnAmplitude2()),
                                    # ("data_channel_swap", ChannelDataSwap()),
                                    # ("postprocessing", PostprocessingTransformer()),
                                    # ("scaler", StandardScaler()),
                    ])

        # ern_features = Pipeline([("feature_extraction", ern_fex)]).transform(ern_features_pre)


        pe_features_pre = Pipeline(steps = [
                                    ("pe_data_extraction", PeTransformer(start_pe_bin=3, stop_pe_bin=8)),
                                    ("pe_amplitude", PeAmplitude2()),
                                    # ("data_channel_swap", ChannelDataSwap()),
                                    # ("postprocessing", PostprocessingTransformer()),
                                    # ("scaler", StandardScaler()),
                    ])

        # pe_features = Pipeline([("feature_extraction", pe_fex)]).transform(pe_features_pre)

    #     x_pre2 = zip(ern_features, pe_features)
    #     x_pre2 = np.array(list(x_pre2)).reshape(x_pre.shape[0],-1)

    #     print(x_pre2.shape)

        ern_pe_features = FeatureUnion([("ern_features", ern_features_pre), ("pe_features", pe_features_pre)], n_jobs = 10)

        x_pre2 = Pipeline([
            ("features", ern_pe_features),
        ]).fit_transform(x_pre_pre)

        # f_vector = np.mean(x_pre2, axis=1)
        f_vector = np.sum(x_pre2, axis=1)

        f_variance = np.std(f_vector)

        data = {'pipeline' : pipeline_name,
                'values' : f_variance,
               }

        within_df = within_df.append(data, ignore_index = True)

### -500 to -300

In [ ]:
X_train_df_500copy = pd.DataFrame(copy.deepcopy(X_train_df_500.to_dict()))

In [ ]:
pre_pip_for_spatial_filter = Pipeline([
        ("channels_extraction",PickChannels(channels_list = box)),
        ("average", Evoked()),
        ('extract_averaged_data', ExtractData()),
        ("spatial_filter_preprocessing", SpatialFilterPreprocessing()),
        ("spatial_filter",PCA(n_components=3, random_state=random_state)),
]).fit(X_train_df_500copy)

spatial_filter = pre_pip_for_spatial_filter['spatial_filter']

In [ ]:
# X_train_df_500copy = pd.DataFrame(copy.deepcopy(X_train_df_500.to_dict()))

In [ ]:
for cutoff in [15,20,30,40]:
    X_train_df_500copy = pd.DataFrame(copy.deepcopy(X_train_df_500.to_dict()))

    pipeline_name = '-500:-300 SF ' + str(cutoff)

    for i in range(0,len(X_train_df_500copy)):

        X = X_train_df_500copy[i:i+1]    
        x_pre = Pipeline([
                ("channels_extraction",PickChannels(channels_list=box)),
                ('extract_data', ExtractDataEpochs()),
        ]).fit_transform(X)  
        x_pre = x_pre[0] 



        x_pre_pre = Pipeline([
            ("spatial_filter_preprocessing", SpatialFilterPreprocessing()),
            ("spatial_filter", spatial_filter),
            ("spatial_filter_postprocessing",SpatialFilterPostprocessing(timepoints_count=181)),
            ("lowpass_filter", LowpassFilter(cutoff=cutoff)),
            ('neg', ReverseComponent3()),
            ("binning", BinTransformer(step=12)),
            ("baseline", ErnBaselined()),
            ("centering", CenteredSignalAfterBaseline3()),
        ]).transform(x_pre)


        ern_features = Pipeline(steps=[
                                    ("ern_data_extraction", ErnTransformer()),
                                    ("ern_amplitude", ErnAmplitude2()),
                                    # ("data_channel_swap", ChannelDataSwap()),
                                    # ("postprocessing", PostprocessingTransformer()),
                                    # ("scaler", StandardScaler()),
                                    # ("feature_extraction", FastICA(random_state=random_state, n_components=3))
                    ])


        pe_features = Pipeline(steps = [
                                    ("pe_data_extraction", PeTransformer(start_pe_bin=3, stop_pe_bin=8)),
                                    ("pe_amplitude", PeAmplitude2()),
                                    # ("data_channel_swap", ChannelDataSwap()),
                                    # ("postprocessing", PostprocessingTransformer()),
                                    # ("scaler", StandardScaler()),
                                    # ("feature_extraction", FastICA(random_state=random_state, n_components=3))
                    ])

        ern_pe_features = FeatureUnion([("ern_features", ern_features), ("pe_features", pe_features)], n_jobs = 10)

        x_pre2 = Pipeline([("features", ern_pe_features)]).fit_transform(x_pre_pre)


    #     x_pre_pre = Pipeline([
    #         ("spatial_filter_preprocessing", SpatialFilterPreprocessing()),
    #         ("spatial_filter", spatial_filter),
    #         ("spatial_filter_postprocessing",SpatialFilterPostprocessing(timepoints_count=181)),
    #         ("lowpass_filter", LowpassFilter()),
    #         ('neg', ReverseComponent3()),
    #         ("binning", BinTransformer(step=12)),
    #         ("baseline", ErnBaselined()),
    #         ("centering", CenteredSignalAfterBaseline3())]).transform(x_pre)


    #     ern_features_pre = Pipeline(steps=[
    #                                 ("ern_data_extraction", ErnTransformer()),
    #                                 ("ern_amplitude", ErnAmplitude2()),
    #                                 ("data_channel_swap", ChannelDataSwap()),
    #                                 ("postprocessing", PostprocessingTransformer()),
    #                                 ("scaler", StandardScaler()),
    #                 ]).fit_transform(x_pre_pre)


    #     ern_features = Pipeline([("feature_extraction", ern_fex)]).transform(ern_features_pre)


    #     pe_features_pre = Pipeline(steps = [
    #                                 ("pe_data_extraction", PeTransformer(start_pe_bin=3, stop_pe_bin=8)),
    #                                 ("pe_amplitude", PeAmplitude2()),
    #                                 ("data_channel_swap", ChannelDataSwap()),
    #                                 ("postprocessing", PostprocessingTransformer()),
    #                                 ("scaler", StandardScaler()),
    #                 ]).fit_transform(x_pre_pre)

    #     pe_features = Pipeline([("feature_extraction", pe_fex)]).transform(pe_features_pre)


    #     x_pre2 = zip(ern_features, pe_features)
    #     x_pre2 = np.array(list(x_pre2)).reshape(x_pre.shape[0],-1)


        # f_vector = np.mean(x_pre2, axis=1)
        f_vector = np.sum(x_pre2, axis=1)     
        f_variance = np.std(f_vector)

        data = {'pipeline' : pipeline_name,
                'values' : f_variance,
               }

        within_df = within_df.append(data, ignore_index = True)
        
        ##################################################################
                
    X_train_df_500copy = pd.DataFrame(copy.deepcopy(X_train_df_500.to_dict()))

    pipeline_name = '-500:-300 SF ' + str(cutoff) + ' no BS'


    for i in range(0,len(X_train_df_500copy)):

        X = X_train_df_500copy[i:i+1]    
        x_pre = Pipeline([
                ("channels_extraction",PickChannels(channels_list=box)),
                ('extract_data', ExtractDataEpochs()),
        ]).fit_transform(X)  
        x_pre = x_pre[0] 



        x_pre_pre = Pipeline([
            ("spatial_filter_preprocessing", SpatialFilterPreprocessing()),
            ("spatial_filter", spatial_filter),
            ("spatial_filter_postprocessing",SpatialFilterPostprocessing(timepoints_count=181)),
            ("lowpass_filter", LowpassFilter(cutoff=cutoff)),
            ('neg', ReverseComponent3()),
            ("binning", BinTransformer(step=12)),
            # ("baseline", ErnBaselined()),
            ("centering", CenteredSignalAfterBaseline3()),
        ]).transform(x_pre)


        ern_features = Pipeline(steps=[
                                    ("ern_data_extraction", ErnTransformer()),
                                    ("ern_amplitude", ErnAmplitude2()),
                                    # ("data_channel_swap", ChannelDataSwap()),
                                    # ("postprocessing", PostprocessingTransformer()),
                                    # ("scaler", StandardScaler()),
                                    # ("feature_extraction", FastICA(random_state=random_state, n_components=3))
                    ])


        pe_features = Pipeline(steps = [
                                    ("pe_data_extraction", PeTransformer(start_pe_bin=3, stop_pe_bin=8)),
                                    ("pe_amplitude", PeAmplitude2()),
                                    # ("data_channel_swap", ChannelDataSwap()),
                                    # ("postprocessing", PostprocessingTransformer()),
                                    # ("scaler", StandardScaler()),
                                    # ("feature_extraction", FastICA(random_state=random_state, n_components=3))
                    ])

        ern_pe_features = FeatureUnion([("ern_features", ern_features), ("pe_features", pe_features)], n_jobs = 10)

        x_pre2 = Pipeline([("features", ern_pe_features)]).fit_transform(x_pre_pre)


    #     x_pre_pre = Pipeline([
    #         ("spatial_filter_preprocessing", SpatialFilterPreprocessing()),
    #         ("spatial_filter", spatial_filter),
    #         ("spatial_filter_postprocessing",SpatialFilterPostprocessing(timepoints_count=181)),
    #         ("lowpass_filter", LowpassFilter()),
    #         ('neg', ReverseComponent3()),
    #         ("binning", BinTransformer(step=12)),
    #         ("baseline", ErnBaselined()),
    #         ("centering", CenteredSignalAfterBaseline3())]).transform(x_pre)


    #     ern_features_pre = Pipeline(steps=[
    #                                 ("ern_data_extraction", ErnTransformer()),
    #                                 ("ern_amplitude", ErnAmplitude2()),
    #                                 ("data_channel_swap", ChannelDataSwap()),
    #                                 ("postprocessing", PostprocessingTransformer()),
    #                                 ("scaler", StandardScaler()),
    #                 ]).fit_transform(x_pre_pre)


    #     ern_features = Pipeline([("feature_extraction", ern_fex)]).transform(ern_features_pre)


    #     pe_features_pre = Pipeline(steps = [
    #                                 ("pe_data_extraction", PeTransformer(start_pe_bin=3, stop_pe_bin=8)),
    #                                 ("pe_amplitude", PeAmplitude2()),
    #                                 ("data_channel_swap", ChannelDataSwap()),
    #                                 ("postprocessing", PostprocessingTransformer()),
    #                                 ("scaler", StandardScaler()),
    #                 ]).fit_transform(x_pre_pre)

    #     pe_features = Pipeline([("feature_extraction", pe_fex)]).transform(pe_features_pre)


    #     x_pre2 = zip(ern_features, pe_features)
    #     x_pre2 = np.array(list(x_pre2)).reshape(x_pre.shape[0],-1)


        # f_vector = np.mean(x_pre2, axis=1)
        f_vector = np.sum(x_pre2, axis=1)     
        f_variance = np.std(f_vector)

        data = {'pipeline' : pipeline_name,
                'values' : f_variance,
               }

        within_df = within_df.append(data, ignore_index = True)

- with SF and FEX

### -100 to 0

In [ ]:
# X_train_df_100copy = pd.DataFrame(copy.deepcopy(X_train_df_100.to_dict()))

In [ ]:
# pre_pip_for_spatial_filter = Pipeline([
#         ("channels_extraction",PickChannels(channels_list = box)),
#         ("average", Evoked()),
#         ('extract_averaged_data', ExtractData()),
#         # ("narrow_indices", NarrowIndices(start=76, stop=257)),
#         ("spatial_filter_preprocessing", SpatialFilterPreprocessing()),
#         ("spatial_filter",PCA(n_components=3, random_state=random_state)),
# ]).fit(X_train_df_100copy)

# spatial_filter = pre_pip_for_spatial_filter['spatial_filter']

In [ ]:
# X_train_df_100copy = pd.DataFrame(copy.deepcopy(X_train_df_100.to_dict()))

In [ ]:
# pipeline_name = '-100:0 SF EX'

# for i in range(0,len(X_train_df_100copy)):
    
#     X = X_train_df_100copy[i:i+1]    
#     x_pre = Pipeline([
#             ("channels_extraction",PickChannels(channels_list=box)),
#             ('extract_data', ExtractDataEpochs()),
#     ]).fit_transform(X)  
    
#     x_pre = x_pre[0] 
    
#     x_pre_pre = Pipeline([
#         ("spatial_filter_preprocessing", SpatialFilterPreprocessing()),
#         ("spatial_filter", spatial_filter),
#         ("spatial_filter_postprocessing",SpatialFilterPostprocessing(timepoints_count=181)),
#         ("lowpass_filter", LowpassFilter()),
#         ('neg', ReverseComponent3()),
#         ("binning", BinTransformer(step=12)),
#         ("baseline", ErnBaselined()),
#         ("centering", CenteredSignalAfterBaseline3())]).transform(x_pre)

    
#     ern_features_pre = Pipeline(steps=[
#                                 ("ern_data_extraction", ErnTransformer()),
#                                 ("ern_amplitude", ErnAmplitude2()),
#                                 ("data_channel_swap", ChannelDataSwap()),
#                                 ("postprocessing", PostprocessingTransformer()),
#                                 # ("scaler", StandardScaler()),
#                 ]).fit_transform(x_pre_pre)
        
#     ern_features = Pipeline([("feature_extraction", ern_fex)]).transform(ern_features_pre)


#     pe_features_pre = Pipeline(steps = [
#                                 ("pe_data_extraction", PeTransformer(start_pe_bin=3, stop_pe_bin=8)),
#                                 ("pe_amplitude", PeAmplitude2()),
#                                 ("data_channel_swap", ChannelDataSwap()),
#                                 ("postprocessing", PostprocessingTransformer()),
#                                 # ("scaler", StandardScaler()),
#                 ]).fit_transform(x_pre_pre)
    
#     pe_features = Pipeline([("feature_extraction", pe_fex)]).transform(pe_features_pre)
        
#     x_pre2 = zip(ern_features, pe_features)
#     x_pre2 = np.array(list(x_pre2)).reshape(x_pre.shape[0],-1)
    
#     # print(x_pre2.shape)

#     # ern_pe_features = FeatureUnion([("ern_features", ern_features), ("pe_features", pe_features)], n_jobs = 10)
    
#     # x_pre2 = Pipeline([
#     #     ("features", ern_pe_features),
#     # ]).fit_transform(x_pre_pre)
    
#     # f_vector = np.mean(x_pre2, axis=1)
#     f_vector = np.sum(x_pre2, axis=1)
    
#     f_variance = np.std(f_vector)
    
#     data = {'pipeline' : pipeline_name,
#             'values' : f_variance,
#            }
    
#     within_df = within_df.append(data, ignore_index = True)
    
    
# #     variances_100_sf.append(f_variance)
    
# # variances_100_sf = np.array(variances_100_sf)

### -500 to -300

In [ ]:
# X_train_df_500copy = pd.DataFrame(copy.deepcopy(X_train_df_500.to_dict()))

In [ ]:
# pre_pip_for_spatial_filter = Pipeline([
#         ("channels_extraction",PickChannels(channels_list = box)),
#         ("average", Evoked()),
#         ('extract_averaged_data', ExtractData()),
#         ("spatial_filter_preprocessing", SpatialFilterPreprocessing()),
#         ("spatial_filter",PCA(n_components=3, random_state=random_state)),
# ]).fit(X_train_df_500copy)

# spatial_filter = pre_pip_for_spatial_filter['spatial_filter']

In [ ]:
# X_train_df_500copy = pd.DataFrame(copy.deepcopy(X_train_df_500.to_dict()))

In [ ]:
# pipeline_name = '-500:-300 SF EX'

# for i in range(0,len(X_train_df_500copy)):
    
#     X = X_train_df_500copy[i:i+1]    
#     x_pre = Pipeline([
#             ("channels_extraction",PickChannels(channels_list=box)),
#             ('extract_data', ExtractDataEpochs()),
#     ]).fit_transform(X)  
#     x_pre = x_pre[0] 

    
    
# #     x_pre_pre = Pipeline([
# #         ("spatial_filter_preprocessing", SpatialFilterPreprocessing()),
# #         ("spatial_filter", spatial_filter),
# #         ("spatial_filter_postprocessing",SpatialFilterPostprocessing(timepoints_count=181)),
# #         ("lowpass_filter", LowpassFilter()),
# #         ('neg', ReverseComponent3()),
# #         ("binning", BinTransformer(step=12)),
# #         ("baseline", ErnBaselined()),
# #         ("centering", CenteredSignalAfterBaseline3()),
# #     ]).transform(x_pre)
    
    
# #     ern_features = Pipeline(steps=[
# #                                 ("ern_data_extraction", ErnTransformer()),
# #                                 ("ern_amplitude", ErnAmplitude2()),
# #                                 ("data_channel_swap", ChannelDataSwap()),
# #                                 ("postprocessing", PostprocessingTransformer()),
# #                                 ("scaler", StandardScaler()),
# #                                 ("feature_extraction", FastICA(random_state=random_state, n_components=3))
# #                 ])


# #     pe_features = Pipeline(steps = [
# #                                 ("pe_data_extraction", PeTransformer(start_pe_bin=3, stop_pe_bin=8)),
# #                                 ("pe_amplitude", PeAmplitude2()),
# #                                 ("data_channel_swap", ChannelDataSwap()),
# #                                 ("postprocessing", PostprocessingTransformer()),
# #                                 ("scaler", StandardScaler()),
# #                                 ("feature_extraction", FastICA(random_state=random_state, n_components=3))
# #                 ])

# #     ern_pe_features = FeatureUnion([("ern_features", ern_features), ("pe_features", pe_features)], n_jobs = 10)
    
# #     x_pre2 = Pipeline([("features", ern_pe_features)]).fit_transform(x_pre_pre)


#     x_pre_pre = Pipeline([
#         ("spatial_filter_preprocessing", SpatialFilterPreprocessing()),
#         ("spatial_filter", spatial_filter),
#         ("spatial_filter_postprocessing",SpatialFilterPostprocessing(timepoints_count=181)),
#         ("lowpass_filter", LowpassFilter()),
#         ('neg', ReverseComponent3()),
#         ("binning", BinTransformer(step=12)),
#         ("baseline", ErnBaselined()),
#         ("centering", CenteredSignalAfterBaseline3())]).transform(x_pre)

    
#     ern_features_pre = Pipeline(steps=[
#                                 ("ern_data_extraction", ErnTransformer()),
#                                 ("ern_amplitude", ErnAmplitude2()),
#                                 ("data_channel_swap", ChannelDataSwap()),
#                                 ("postprocessing", PostprocessingTransformer()),
#                                 # ("scaler", StandardScaler()),
#                 ]).fit_transform(x_pre_pre)
    
    
#     ern_features = Pipeline([("feature_extraction", ern_fex)]).transform(ern_features_pre)


#     pe_features_pre = Pipeline(steps = [
#                                 ("pe_data_extraction", PeTransformer(start_pe_bin=3, stop_pe_bin=8)),
#                                 ("pe_amplitude", PeAmplitude2()),
#                                 ("data_channel_swap", ChannelDataSwap()),
#                                 ("postprocessing", PostprocessingTransformer()),
#                                 # ("scaler", StandardScaler()),
#                 ]).fit_transform(x_pre_pre)
    
#     pe_features = Pipeline([("feature_extraction", pe_fex)]).transform(pe_features_pre)
    
    
#     x_pre2 = zip(ern_features, pe_features)
#     x_pre2 = np.array(list(x_pre2)).reshape(x_pre.shape[0],-1)
    

#     # f_vector = np.mean(x_pre2, axis=1)
#     f_vector = np.sum(x_pre2, axis=1)     
#     f_variance = np.std(f_vector)
    
#     data = {'pipeline' : pipeline_name,
#             'values' : f_variance,
#            }
    
#     within_df = within_df.append(data, ignore_index = True)
# #     variances_500_sf.append(f_variance)
    
# # variances_500_sf = np.array(variances_500_sf)

### Visualization

In [ ]:
import seaborn as sns
sns.set(rc={'figure.figsize':(10,12)})
sns.set_theme(style="whitegrid")


ax = sns.violinplot(x = 'values', y = 'pipeline', data = within_df, orient='h', )
# ax.figure.savefig("within_subject_std_ern_pe_with_lowpass_BS.png")

In [ ]:
import seaborn as sns
sns.set(rc={'figure.figsize':(10,12)})
sns.set_theme(style="whitegrid")


ax = sns.violinplot(x = 'values', y = 'pipeline', data = within_df, orient='h', )
# ax.figure.savefig("within_subject_std_ern_pe_with_lowpass_BS.png")

## Internal consistency

consistency = betweenPerson / between_person + within_person

In [ ]:
internal_df = pd.DataFrame({'pipeline': [], 'internal_variance': []})

for pipeline in between_df['pipeline'].unique().tolist():
    
    between_std = np.std(np.array(between_df.loc[between_df['pipeline'] == pipeline, 'values'].tolist()))                     
    within_list = np.array(within_df.loc[within_df['pipeline'] == pipeline, 'values'].tolist())
    
    for person_variance in within_list:
        
        internal = between_std/(between_std + person_variance)    
        data = {'pipeline' : pipeline,
                'internal_variance' : internal,
               }
    
        internal_df = internal_df.append(data, ignore_index = True)

In [ ]:
import seaborn as sns
sns.set(rc={'figure.figsize':(10,8)})
sns.set_theme(style="whitegrid")


ax = sns.violinplot(x = 'internal_variance', y = 'pipeline', data = internal_df, orient='h', inner="quartile")
# ax.figure.savefig("internal_consistency_ern_pe_lowpass_BS.png")

In [ ]:
within_df[(within_df['pipeline'] == '-100:0 SF') & (within_df['values'] > 0.00005)].index

In [ ]:
# 182 == 0

indexes = within_df[(within_df['pipeline'] == '-100:0 SF') & (within_df['values'] > 0.00005)].index
indexes = np.array(indexes.tolist())

indexes_new = [index - 196 for index in indexes]
indexes_new
# index 8, 19, 33, 

---
# Visualization of Components

In [ ]:
import copy
X_train_df_copy = pd.DataFrame(copy.deepcopy(X_train_df_100.to_dict()))
# X_test_df_copy = pd.DataFrame(copy.deepcopy(X_test_df.to_dict()))

In [ ]:
box = ['Fpz', 'AFz', 'Fz', 'FCz', 'C1', 'Cz', 'C2', 'CPz', 'P1', 'Pz', 'P2']
red_box8_prim = [
    "Fpz", 
    "AFz",
    "Fz",
    "FCz",
    "C1", "Cz","C2",
    "CPz",
    "P1", "Pz", "P2",
]

In [ ]:
pre_pip = Pipeline([
        ("channels_extraction",PickChannels(channels_list = red_box8_prim)),
        ("average", Evoked()),
        ('extract_averaged_data', ExtractData()),
        # ("narrow_indices", NarrowIndices(start=76, stop=257)),
        ("spatial_filter_preprocessing", SpatialFilterPreprocessing()),
        ("spatial_filter",PCA(n_components=3, random_state=random_state)),
        ("spatial_filter_postprocessing",SpatialFilterPostprocessing(timepoints_count=181)),
        ("lowpass_filter", LowpassFilter()),
        ('neg', ReverseComponent3()),
        ("binning", BinTransformer(step=12)),
        # ("baseline", ErnBaselined()),
        ("centering", CenteredSignalAfterBaseline3()),
]).fit(X_train_df_copy)

X = pre_pip.transform(X_train_df_copy)
X_mean = np.mean(X, axis=0)

In [ ]:
X_mean.shape

In [ ]:
sf = pre_pip['spatial_filter']

In [ ]:
components = sf.components_

In [ ]:
def check_positive(number):
    if number > 0:
          return True  

    return False

def check_negative(number):
    if number < 0:
          return True  

    return False

In [ ]:
components_copy = components.copy()

In [ ]:
components_copy

In [ ]:
components_1 = [components_copy[1]]
components_2 = [components_copy[2]]

In [ ]:
components_2

In [ ]:
for component in components_1:
    print(component)

    positive = []
    negative = []
    for item in component:
        if item > 0:
            positive.append(item)
            negative.append(0)
        else:
            positive.append(0)
            negative.append(item)
    print(positive)
    print(negative)
    
positive = np.array(positive).reshape(11,-1)
negative = np.array(negative).reshape(11,-1)

positive_compo = X_mean * positive
negative_compo = X_mean * negative

positive_signal_1 = np.sum(positive_compo, axis=0)
negative_signal_1 = np.sum(negative_compo, axis=0)

In [ ]:
for component in components_2:
    print(component)

    positive = []
    negative = []
    for item in component:
        if item > 0:
            positive.append(item)
            negative.append(0)
        else:
            positive.append(0)
            negative.append(item)
    print(positive)
    print(negative)
    
positive = np.array(positive).reshape(11,-1)
negative = np.array(negative).reshape(11,-1)

positive_compo = X_mean * positive
negative_compo = X_mean * negative

positive_signal_2 = np.sum(positive_compo, axis=0)
negative_signal_2 = np.sum(negative_compo, axis=0)

In [ ]:
c_1 = positive_signal_1 + negative_signal_1
c_2 = positive_signal_2 + negative_signal_2

In [ ]:
import matplotlib.pyplot as plt

plt.plot(positive_signal_2)
plt.plot(-negative_signal_2)

# plt.plot(X_mean[1], lw=4)
plt.plot(c_2, lw = 4)
# plt.savefig("differences_component_3")

In [ ]:
import matplotlib.pyplot as plt

for i in range(0,3):
    plt.plot(X_mean[i])

In [ ]:
import matplotlib.pyplot as plt

for i in range(0,10):
    plt.plot(X[i][0])

In [ ]:
import matplotlib.pyplot as plt

for i in range(0,10):
    plt.plot(X[i][1])

In [ ]:
import matplotlib.pyplot as plt

for i in range(0,10):
    plt.plot(X[i][2])